**Important: This notebook will only work with fastai-0.7.x. Do not try to run any fastai-1.x code from this path in the repository because it will load fastai-0.7.x**

# Neto customer churn predict project

## Summary

This is a data project to see if we can take customer data from the Neto platform and categorise and hopefully in the end "predict" when a customer is likely to churn. There will be a heap of assumptions which I will note in here.

### Research links

https://www.kaggle.com/pavanraj159/telecom-customer-churn-prediction/data

### Assumptions

- churn can be indicated by last login date of an account
- usage behaviour of a customers use of the export system could indicate signs of churn

### Data observation

`Neto License Registration Date` and `Go Live Date` are sometime entered "backwards" so when calulating the `go live days` I am taking the absolute value so I dont get a negative value.

`go live day` - if they are not `LIVE` I set the value to `-1` so we know its not live yet, 0 is a valid period of days.

`canceldate` - is not always in the future so I have had to get absolute values when calculating days active

## Imports

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

%matplotlib inline

In [2]:
import numpy as np
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display


from sklearn import metrics

In [3]:
PATH = "../../data/"

In [4]:
!ls {PATH}

Data Dictionary.xlsx             churn.csv
Machine_Appendix.csv             human_numbers.tgz
Test.csv                         median_benchmark.csv
adult_sample.tgz                 random_forest_benchmark_test.csv
bulldozers                       test


In [5]:
df_raw = pd.read_csv(f'{PATH}churn.csv', low_memory=False, 
                     parse_dates=["canceldate", "licence_registration_date", "golive_date"])

In [6]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [7]:
df_raw.describe(include='all')

,username,merchant,canceldate,licence_registration_date,days_active,last_active_days,account_status,nps,licence_status,cancel_reason,...,plans,plan,changing_platform,new_platform,salesperson,golive_date,golive_days,url,churned,churn_concern
count,5889,5885,5889,5889,5889.000000,3121.000000,3128,513.000000,5889,3111,...,5884,5884,1821,738,5889,5889,5889.000000,5884,5889.000000,3276.000000
unique,5889,5778,913,1529,NaN,NaN,5,NaN,4,18,...,42,42,2,121,26,936,NaN,5852,NaN,NaN
top,N001086,Invertek,1970-01-01 00:00:00,2013-11-25 00:00:00,NaN,NaN,Active,NaN,CANCELLED,Non-payment (Neto),...,PLAN-S-19Q1,PLAN-S-19Q1,No,Shopify,rene,1970-01-01 00:00:00,NaN,www.southerncrossemporium.com.au,NaN,NaN
freq,1,5,2776,126,NaN,NaN,2643,NaN,3109,621,...,1530,1530,1073,189,943,2706,NaN,4,NaN,NaN
first,NaN,NaN,1970-01-01 00:00:00,2010-07-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1970-01-01 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,2019-09-06 00:00:00,2019-09-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019-06-12 00:00:00,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,673.074036,26.166293,NaN,4.337232,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,67.630328,NaN,0.527933,0.954823
std,NaN,NaN,NaN,NaN,561.863296,59.673295,NaN,3.536268,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,142.993069,NaN,0.499262,0.207724
min,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,191.000000,0.000000,NaN,1.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,0.000000,1.000000


### Initial processing

This dataset contains a mix of **continuous** and **categorical** variables.

The following method extracts particular date fields from a complete datetime for the purpose of constructing categoricals.  You should always consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities.

In [8]:
# These steps can only be run once then you need to comment them out

#add_datepart(df_raw, 'canceldate')
add_datepart(df_raw, 'licence_registration_date')
add_datepart(df_raw, 'golive_date')

The code above breaks up all of the dates fields into a bunch of different fields, displayed below.

The categorical variables are currently stored as strings, which is inefficient, and doesn't provide the numeric coding required for a random forest. Therefore we call `train_cats` to convert strings to pandas categories.

In [9]:
df_raw = df_raw.drop(columns=['licence_status','account_status', 'canceldate', 'cancel_details', 'cancel_reason', 'changing_platform'])

In [10]:
train_cats(df_raw)

This fills all NaN values in the follow fields with that columns mean.

In [11]:
df_raw.nps = df_raw.nps.fillna(np.nanmean(df_raw.nps))
df_raw.last_active_days = df_raw.last_active_days.fillna(np.mean(df_raw.last_active_days))
df_raw.churn_concern = df_raw.churn_concern.fillna(0)
display_all(df_raw)

,username,merchant,days_active,last_active_days,nps,team,vertical,plans,plan,new_platform,salesperson,golive_days,url,churned,churn_concern,licence_registration_Year,licence_registration_Month,licence_registration_Week,licence_registration_Day,licence_registration_Dayofweek,licence_registration_Dayofyear,licence_registration_Is_month_end,licence_registration_Is_month_start,licence_registration_Is_quarter_end,licence_registration_Is_quarter_start,licence_registration_Is_year_end,licence_registration_Is_year_start,licence_registration_Elapsed,golive_Year,golive_Month,golive_Week,golive_Day,golive_Dayofweek,golive_Dayofyear,golive_Is_month_end,golive_Is_month_start,golive_Is_quarter_end,golive_Is_quarter_start,golive_Is_year_end,golive_Is_year_start,golive_Elapsed
0,NoveltyShop,The Novelty Shop,1851,26.166293,4.337232,not specified,not specified,BUSINESS ONE,BUSINESS ONE,NaN,dominic,-1,NaN,1,1.0,2010,7,26,3,5,184,False,False,False,False,False,False,1278115200,1970,1,1,1,3,1,False,True,False,True,False,True,0
1,N002341,Mx Store,2251,0.000000,4.337232,Neto XL,Sports & Recreation,PLAN-XL-19Q1,PLAN-XL-19Q1,NaN,not specified,-1,www.mxstore.com.au,0,0.0,2013,7,29,16,1,197,False,False,False,False,False,False,1373932800,1970,1,1,1,3,1,False,True,False,True,False,True,0
2,N001714,Just Health,2250,1.000000,7.000000,not specified,Health & Beauty,PLAN-S-19Q1,PLAN-S-19Q1,NaN,cat,-1,www.just-health.com.au,0,0.0,2013,7,29,17,2,198,False,False,False,False,False,False,1374019200,1970,1,1,1,3,1,False,True,False,True,False,True,0
3,N002628,Barefoot Healing Pty Ltd,2250,0.000000,4.337232,not specified,Health & Beauty,PLAN-L-19Q1,PLAN-L-19Q1,NaN,not specified,-1,www.barefoothealing.com.au,0,0.0,2013,7,29,17,2,198,False,False,False,False,False,False,1374019200,1970,1,1,1,3,1,False,True,False,True,False,True,0
4,N002758,Aussie Fitness Equipment,2195,0.000000,4.337232,not specified,Sports & Recreation,PLAN-L-19Q1,PLAN-L-19Q1,NaN,rene,-1,www.aussiefitness.com.au,0,0.0,2013,9,37,10,1,253,False,False,False,False,False,False,1378771200,1970,1,1,1,3,1,False,True,False,True,False,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5884,N065838,Thryve Nutrition,10,0.000000,4.337232,Team Imagine,Supplements & Nutrit,PLAN-M-19Q1,PLAN-M-19Q1,NaN,simon.white,-1,thryve-nutritio.neto.com.au,0,0.0,2019,9,36,4,2,247,False,False,False,False,False,False,1567555200,1970,1,1,1,3,1,False,True,False,True,False,True,0
5885,N066053,McMullin & co,10,0.000000,4.337232,Team Vision,Homewares,PLAN-M-19Q1,PLAN-M-19Q1,NaN,simon.white,-1,mcmullinandco.neto.com.au,0,0.0,2019,9,36,4,2,247,False,False,False,False,False,False,1567555200,1970,1,1,1,3,1,False,True,False,True,False,True,0
5886,N065608,McsOnline,9,0.000000,4.337232,Neto XL,Motor Parts,PLAN-S-19Q1,PLAN-S-19Q1,NaN,simon.white,-1,mcsonline.neto.com.au,0,0.0,2019,9,36,5,3,248,False,False,False,False,False,False,1567641600,1970,1,1,1,3,1,False,True,False,True,False,True,0
5887,N066090,MX Online Australia 2,9,0.000000,4.337232,Neto XL,Motor Parts,PLAN-L-19Q1,PLAN-L-19Q1,NaN,simon.white,-1,mx-online-austr-1.neto.com.au,0,0.0,2019,9,36,5,3,248,False,False,False,False,False,False,1567641600,1970,1,1,1,3,1,False,True,False,True,False,True,0


But let's save this file for now, since it's already in format can we be stored and accessed efficiently.

In [12]:
df_raw.columns

Index(['username', 'merchant', 'days_active', 'last_active_days', 'nps',
       'team', 'vertical', 'plans', 'plan', 'new_platform', 'salesperson',
       'golive_days', 'url', 'churned', 'churn_concern',
       'licence_registration_Year', 'licence_registration_Month',
       'licence_registration_Week', 'licence_registration_Day',
       'licence_registration_Dayofweek', 'licence_registration_Dayofyear',
       'licence_registration_Is_month_end',
       'licence_registration_Is_month_start',
       'licence_registration_Is_quarter_end',
       'licence_registration_Is_quarter_start',
       'licence_registration_Is_year_end',
       'licence_registration_Is_year_start', 'licence_registration_Elapsed',
       'golive_Year', 'golive_Month', 'golive_Week', 'golive_Day',
       'golive_Dayofweek', 'golive_Dayofyear', 'golive_Is_month_end',
       'golive_Is_month_start', 'golive_Is_quarter_end',
       'golive_Is_quarter_start', 'golive_Is_year_end', 'golive_Is_year_start',
       'goli

In [13]:
df, y, nas = proc_df(df_raw, 'churned', max_n_cat=5)
display_all(df)

,username,merchant,days_active,last_active_days,nps,team,vertical,plans,plan,new_platform,salesperson,golive_days,url,churn_concern,licence_registration_Year,licence_registration_Month,licence_registration_Week,licence_registration_Day,licence_registration_Dayofweek,licence_registration_Dayofyear,licence_registration_Is_month_end,licence_registration_Is_month_start,licence_registration_Is_quarter_end,licence_registration_Is_quarter_start,licence_registration_Is_year_end,licence_registration_Is_year_start,licence_registration_Elapsed,golive_Year,golive_Month,golive_Week,golive_Day,golive_Dayofweek,golive_Dayofyear,golive_Is_month_end,golive_Is_month_start,golive_Is_quarter_end,golive_Is_quarter_start,golive_Is_year_end,golive_Is_year_start,golive_Elapsed
0,5750,4978,1851,26.166293,4.337232,14,26,5,5,0,11,-1,0,1.0,2010,7,26,3,5,184,False,False,False,False,False,False,1278115200,1970,1,1,1,3,1,False,True,False,True,False,True,0
1,256,3300,2251,0.000000,4.337232,4,20,33,33,0,18,-1,4405,0.0,2013,7,29,16,1,197,False,False,False,False,False,False,1373932800,1970,1,1,1,3,1,False,True,False,True,False,True,0
2,222,2612,2250,1.000000,7.000000,14,8,31,31,0,8,-1,3971,0.0,2013,7,29,17,2,198,False,False,False,False,False,False,1374019200,1970,1,1,1,3,1,False,True,False,True,False,True,0
3,273,612,2250,0.000000,4.337232,14,8,23,23,0,18,-1,2686,0.0,2013,7,29,17,2,198,False,False,False,False,False,False,1374019200,1970,1,1,1,3,1,False,True,False,True,False,True,0
4,285,434,2195,0.000000,4.337232,14,20,23,23,0,20,-1,2605,0.0,2013,9,37,10,1,253,False,False,False,False,False,False,1378771200,1970,1,1,1,3,1,False,True,False,True,False,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5884,5729,5058,10,0.000000,4.337232,8,21,27,27,0,24,-1,2143,0.0,2019,9,36,4,2,247,False,False,False,False,False,False,1567555200,1970,1,1,1,3,1,False,True,False,True,False,True,0
5885,5746,3090,10,0.000000,4.337232,9,10,27,27,0,24,-1,1246,0.0,2019,9,36,4,2,247,False,False,False,False,False,False,1567555200,1970,1,1,1,3,1,False,True,False,True,False,True,0
5886,5712,3091,9,0.000000,4.337232,4,15,31,31,0,24,-1,1249,0.0,2019,9,36,5,3,248,False,False,False,False,False,False,1567641600,1970,1,1,1,3,1,False,True,False,True,False,True,0
5887,5747,2978,9,0.000000,4.337232,4,15,23,23,0,24,-1,1343,0.0,2019,9,36,5,3,248,False,False,False,False,False,False,1567641600,1970,1,1,1,3,1,False,True,False,True,False,True,0


In [14]:
# This is how to save the current data set

os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/churn')

### Pre-processing

In the future we can simply read it from this fast format.

In [15]:
df_raw = pd.read_feather('tmp/churn')

We'll replace categories with their numeric codes, handle missing continuous values, and split the dependent variable into a separate variable.

In [110]:
df, y, nas = proc_df(df_raw, 'churned')

In [126]:
display_all(df)

,username,merchant,days_active,last_active_days,is_suspended,nps,cancel_reason,cancel_details,team,vertical,plans,plan,changing_platform,new_platform,salesperson,golive_days,url,churn_concern,cancelYear,cancelMonth,cancelWeek,cancelDay,cancelDayofweek,cancelDayofyear,cancelIs_month_end,cancelIs_month_start,cancelIs_quarter_end,cancelIs_quarter_start,cancelIs_year_end,cancelIs_year_start,cancelElapsed,licence_registration_Year,licence_registration_Month,licence_registration_Week,licence_registration_Day,licence_registration_Dayofweek,licence_registration_Dayofyear,licence_registration_Is_month_end,licence_registration_Is_month_start,licence_registration_Is_quarter_end,licence_registration_Is_quarter_start,licence_registration_Is_year_end,licence_registration_Is_year_start,licence_registration_Elapsed,golive_Year,golive_Month,golive_Week,golive_Day,golive_Dayofweek,golive_Dayofyear,golive_Is_month_end,golive_Is_month_start,golive_Is_quarter_end,golive_Is_quarter_start,golive_Is_year_end,golive_Is_year_start,golive_Elapsed,is_suspended_na
0,5750,4978,1851,26.166293,0.0,4.337232,6,0,14,26,5,5,0,0,11,-1,0,0.0,2015,7,31,28,1,209,False,False,False,False,False,False,1438041600,2010,7,26,3,5,184,False,False,False,False,False,False,1278115200,1970,1,1,1,3,1,False,True,False,True,False,True,0,True
1,256,3300,2248,0.000000,0.0,4.337232,0,0,4,20,33,33,0,0,18,-1,4405,0.0,1970,1,1,1,3,1,False,True,False,True,False,True,0,2013,7,29,16,1,197,False,False,False,False,False,False,1373932800,1970,1,1,1,3,1,False,True,False,True,False,True,0,False
2,222,2612,2247,1.000000,0.0,7.000000,0,0,14,8,31,31,0,0,8,-1,3971,0.0,1970,1,1,1,3,1,False,True,False,True,False,True,0,2013,7,29,17,2,198,False,False,False,False,False,False,1374019200,1970,1,1,1,3,1,False,True,False,True,False,True,0,False
3,273,612,2247,0.000000,0.0,4.337232,0,0,14,8,23,23,0,0,18,-1,2686,0.0,1970,1,1,1,3,1,False,True,False,True,False,True,0,2013,7,29,17,2,198,False,False,False,False,False,False,1374019200,1970,1,1,1,3,1,False,True,False,True,False,True,0,False
4,285,434,2192,0.000000,0.0,4.337232,0,0,14,20,23,23,0,0,20,-1,2605,0.0,1970,1,1,1,3,1,False,True,False,True,False,True,0,2013,9,37,10,1,253,False,False,False,False,False,False,1378771200,1970,1,1,1,3,1,False,True,False,True,False,True,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5884,5729,5058,7,0.000000,0.0,4.337232,0,0,8,21,27,27,0,0,24,-1,2143,0.0,1970,1,1,1,3,1,False,True,False,True,False,True,0,2019,9,36,4,2,247,False,False,False,False,False,False,1567555200,1970,1,1,1,3,1,False,True,False,True,False,True,0,False
5885,5746,3090,7,0.000000,0.0,4.337232,0,0,9,10,27,27,0,0,24,-1,1246,0.0,1970,1,1,1,3,1,False,True,False,True,False,True,0,2019,9,36,4,2,247,False,False,False,False,False,False,1567555200,1970,1,1,1,3,1,False,True,False,True,False,True,0,False
5886,5712,3091,6,0.000000,0.0,4.337232,0,0,4,15,31,31,0,0,24,-1,1249,0.0,1970,1,1,1,3,1,False,True,False,True,False,True,0,2019,9,36,5,3,248,False,False,False,False,False,False,1567641600,1970,1,1,1,3,1,False,True,False,True,False,True,0,False
5887,5747,2978,6,0.000000,0.0,4.337232,0,0,4,15,23,23,0,0,24,-1,1343,0.0,1970,1,1,1,3,1,False,True,False,True,False,True,0,2019,9,36,5,3,248,False,False,False,False,False,False,1567641600,1970,1,1,1,3,1,False,True,False,True,False,True,0,False
